In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datasets import load_dataset

dataset = load_dataset("LeMaterial/LeMat-Synth")

dataset

In [ ]:
dataset_papers = load_dataset(
    "LeMaterial/LeMat-Synth-Papers", split="sample_for_evaluation"
)

dataset_papers

In [ ]:
df_papers = dataset_papers.to_pandas()

df_papers

In [ ]:
ds = dataset["sample_for_evaluation"]

ds

In [ ]:
df = ds.to_pandas()

df

In [ ]:
# give doi of row where id=2212.12506
doi = df_papers[df_papers["id"] == "2212.12506"]["doi"]
doi = doi.iloc[0]

# give index of row in df where paper_doi=doi
sample = df[df["paper_doi"] == doi].iloc[0]

# give row where index=idx
sample

In [ ]:
# sample = df.iloc[0]

# sample

In [ ]:
from IPython.display import Markdown, display

from llm_synthesis.transformers.figure_extraction.hf_figure_extractor import (
    HFFigureExtractor,
)

hf_figure_extractor = HFFigureExtractor()

segmented_images = hf_figure_extractor.forward(sample["images"])
print(f"Found {len(segmented_images)} figures in the paper.")
quantative_images = [img for img in segmented_images if img.quantitative]
print(f"Found {len(quantative_images)} quantitative figures in the paper.")

for quantative_image in quantative_images:
    display(
        Markdown(
            f"![Subfigure](data:image/png;base64,{quantative_image.base64_data})"
        )
    )
    print(quantative_image.figure_class)
    print("-" * 100)

In [ ]:
for image in segmented_images:
    display(
        Markdown(f"![Subfigure](data:image/png;base64,{image.base64_data})")
    )
    print(image.figure_class)
    print("-" * 100)

In [ ]:
from llm_synthesis.transformers.plot_extraction.claude_extraction.plot_data_extraction import (
    ClaudeLinePlotDataExtractor,
)
from llm_synthesis.utils.visualization import visualize_chart

# Initialize the extractor
extractor = ClaudeLinePlotDataExtractor(model_name="claude-sonnet-4-20250514")

# Perform the extraction
extracted_data = extractor.forward(quantative_images[0])
print("Extracted data:")
print(extracted_data)

# Track the cost of this operation
cost_info = extractor.get_cost()
cost_info

In [ ]:
for quantative_image in quantative_images:
    figure_class = quantative_image.figure_class
    print(figure_class)
    display(
        Markdown(
            f"![Subfigure](data:image/png;base64,{quantative_image.base64_data})"
        )
    )
    extracted_data = extractor.forward(quantative_image)
    visualize_chart(extracted_data, figure_class=figure_class)
    print("-" * 100)